In [1]:
import os
from sqlalchemy import create_engine
# from sqlalchemy.orm import sessionmaker

def db_connect():
    if 'DB_USER_FA' in os.environ:
        DB_USER = os.environ['DB_USER_FA']
        DB_PASSWORD = os.environ['DB_PASSWORD_FA']
        DB_SERVER = "w4111.cisxo09blonu.us-east-1.rds.amazonaws.com"
        DATABASEURI = "postgresql://"+DB_USER+":"+DB_PASSWORD+"@"+DB_SERVER+"/w4111"
        return create_engine(DATABASEURI)
    else:
        raise Exception(
            'Database Credentials Not Found in the Environment Variables')

In [2]:
os.environ['DB_USER_FA']

'alm2263'

In [56]:
hi,_,_ = ("asdf","df","f")

In [57]:
hi

'asdf'

In [58]:
_

'f'

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet
# from db import db_connect


class StockAPI(object):
    def __init__(self):
        self.engine = db_connect()
        print("connected to db...")

    def recommend(self):
        """
        find top 5 popular stock according to the watchlist
        """
        
        s="""
        select S.name, COUNT(*)
        from stock S, Watchlist W
        where S.ticker = W.ticker
        group by S.ticker
        order by count(*) desc
        limit 5;
        """
        cursor = self.engine.execute(s)
        names = []
        for result in cursor:
            names.append(result['name'])  # can also be accessed using result[0]
        cursor.close()
        stock_str = ", ".join(names)
        return "here is top 5 popular of stocks: {} ".format(stock_str)

    def search_stock(self, stock):
        """
        find the latest price of the stock
        """

        s="""select * 
        from Tick as T join Stock as S on T.ticker = S.ticker
        where S.name ~* %s
        order by record_date desc
        limit 1
        """
        cursor = self.engine.execute(s, (stock,))
        for result in cursor:
            date, open_price, close_price,_,_,_,_ = result
        cursor.close()
        return "the open price for {} at {} is {} and the close price is {}".format(stock,
                                                                                    str(date),
                                                                                   open_price,
                                                                                   close_price)

In [7]:
stock_api = StockAPI()

connected to db...


In [26]:
s="""select * 
        from Tick as T join Stock as S on T.ticker = S.ticker
        where S.name ~* %s
        order by record_date desc
        limit 1
        """
c=stock_api.engine.execute(s,("walmt"))
c.fetchone()

In [27]:
c.fetchone()

In [30]:
s="""select name 
        from Stock
        """
c=stock_api.engine.execute(s)

In [31]:
r=c.fetchall()

In [32]:
r

[('Walmart',),
 ('Exxon Mobil',),
 ('Berkshire Hathaway',),
 ('Apple',),
 ('UnitedHealth Group',),
 ('McKesson',),
 ('CVS Health',),
 ('Amazon',),
 ('AT&T',),
 ('General Motors',),
 ('JPMorgan Chase',),
 ('Boeing',),
 ('Anheuser-Busch',)]

In [34]:
# c.fetchall()
c.close()

In [28]:
c.close()

In [15]:
for c_r in c:
    nothing=c_r
c.close()

In [16]:
if nothing:
    print("yest")

NameError: name 'nothing' is not defined

In [4]:
stock_api.search_stock("Walmart")

'the open price for Walmart at 2018-10-22 is 97.34 and the close price is 96.74'

In [9]:
stock_api.search_stock("what?")

UnboundLocalError: local variable 'date' referenced before assignment

In [51]:
con = stock_api.engine.connect()

In [52]:
ss="""select * 
        from Tick as T join Stock as S on T.ticker = S.ticker
        where S.name = %s
        order by record_date desc
        limit 1
        """

In [53]:
ss

'select * \n        from Tick as T join Stock as S on T.ticker = S.ticker\n        where S.name = %s\n        order by record_date desc\n        limit 1\n        '

In [59]:
re=con.execute(ss,("Walmart",))
for r in re:
    date, open_price, close_price,_,_,_,_ = r
re.close()

In [70]:
from datetime import datetime

In [76]:
str(date)

'2018-10-22'

In [25]:
re=con.execute("""SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='alm2263'""")
for row in re:
    print (row)

In [18]:
result.close()

In [ ]:
class ActionSearchStock(Action):
    """
    given a stock name or ticker name,
    provide the latest price and relevant information
    """
    def name(self):
        return 'action_search_stock'

    def run(self, dispatcher, tracker, domain):
        # dispatcher.utter_message("looking for this stock")
        stock_info = stock_api.search(tracker.get_slot("stock"))
        dispatcher.utter_message("{}".format(stock_info))
        return []


class ActionSuggestStock(Action):
    def name(self):
        return 'action_suggest_stock'

    def run(self, dispatcher, tracker, domain):
        recommend_info = stock_api.recommend()
        dispatcher.utter_message(
          "here's what I found: {}".format(recommend_info))
        return []